In [6]:
include("lib/all.jl");

In [8]:
test_acc()

839685

In [9]:
particles = make_particles(2^20);
t = Tree(particles, 16);

In [ ]:
@time group!(t);

In [11]:
@time inform!(t);

  0.151118 seconds (879.20 k allocations: 14.647 MB)


In [12]:
test_in_cell_mass(t)

In [29]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [34]:
@time interact!(t, 0.5, ax, ay, az);

  1.810878 seconds (4.81 M allocations: 1.506 GB, 1.86% gc time)
